https://www.epj-conferences.org/articles/epjconf/pdf/2023/09/epjconf_uhecr2023_04009.pdf

## Bellhop - Acoustic Toolbox

# Sound Speed Profile of Tanganyika site

In [ ]:
import arlpy.uwapm as pm
import arlpy.plot as plt
import numpy as np

env = pm.create_env2d()

# make the speed of sound profile for the Tanganyika site
ssp_T = [
    [   0, 1503.89725749], 
    [   5, 1503.53512228],
    [  27, 1503.40633872],
    [  45, 1501.17598527],
    [  63, 1497.95739375],
    [  91, 1496.57667752],
    [ 120, 1496.18094787],
    [ 144, 1496.13991023],
    [ 168, 1496.23668012],
    [ 187, 1496.33233254],
    [ 216, 1496.82067334],
    [ 250, 1497.30878916],
    [ 300, 1497.98142059],
    [ 350, 1498.79460352],
    [ 400, 1499.46622159], 
    [ 450, 1500.36382161],
    [ 625, 1503.19287932],
    [ 721, 1504.83479986], 
    [ 818, 1506.35078556],
    [ 867, 1507.14541936],
    [1500, 1517.77346984]]   

# frequency min and max: https://www.sciencedirect.com/science/article/pii/S0168900212015239
frequency_min = 25000      # 25 kHz
frequency_max = 45000      # 45 kHz

In [ ]:
# Create the 2D environment with both the source and the receiver placed 3 m above the sea floor
env = pm.create_env2d(frequency=frequency_min, min_angle=-89, max_angle=89, depth=1500, tx_depth=1497, rx_depth=1400, rx_range=10000, soundspeed=ssp_T, nbeams=10000)

# Plot the sound speed profile
pm.plot_ssp(env, width=500)
pm.plot_env(env, width=1000)

In [ ]:
rays = pm.compute_rays(env)
rays = pm.plot_rays(rays, env=env, width=900)

# Scan of distances and elevations

In [ ]:
# define the range of distance and elevation you want to research
distance = np.linspace(50, 10000, 200)
elevation = np.linspace(0, 1000, 11)

print(distance)
print(elevation)

In [ ]:
# initiate the lists that will be filled whith eigenray properties for each of the to be looped through situations
distance_sea_floor = []
height_DOM = []
arrival_time = []
angle_of_departure = []
angle_of_arrival = []

# loop through the distances and elevations to check for each scenario individually
for i in range(0, len(distance)):
    for j in range(0, len(elevation)):
        dis = distance[i]
        el = elevation[j]

        if el == 0:
            el = 3

        if dis == 0:
            dis = 13
        
        #print(f"dis: {dis}")
        #print(f"el: {el}")
        
        # define the depth of the ocean at the location of your transmitter
        transmitter_depth = 1497
        receiver_depth = (1500 - el)
        tx_depth = transmitter_depth - 3
        rx_depth = receiver_depth
        #print(f"receiver_depth: {receiver_depth}")
        #print(f"tx_depth: {tx_depth}")
        #print(f"rx_depth: {rx_depth}")
        
        # create two environments with different frequencies
        env_min = pm.create_env2d(frequency=frequency_min, min_angle=-89, max_angle=89, depth=1500, tx_depth=tx_depth, rx_depth=rx_depth, rx_range=dis, soundspeed=ssp_T, nbeams=100000)
        env_max = pm.create_env2d(frequency=frequency_max, min_angle=-89, max_angle=89, depth=1500, tx_depth=tx_depth, rx_depth=rx_depth, rx_range=dis, soundspeed=ssp_T, nbeams=100000)

        #pm.plot_env(env_min, surface_color='dodgerblue', bottom_color='peru', rx_color='blue', rx_plot=None, width=900)
        
        if (dis != 0) or (el != 0):
            try:
                # compute the arrivals
                arrivals_min = pm.compute_arrivals(env_min)
                arrivals_max = pm.compute_arrivals(env_max)
    
                # check which arrivals have 0 bounces against the surface or the bottom
                zero_bounces_min = arrivals_min[(arrivals_min['surface_bounces'] == 0) & (arrivals_min['bottom_bounces'] == 0)]
                zero_bounces_max = arrivals_max[(arrivals_max['surface_bounces'] == 0) & (arrivals_max['bottom_bounces'] == 0)]
    
                # count how many rays arrive with no bounces and save necessary quantities to list
                for time in zero_bounces_min['time_of_arrival']:
                    height_DOM.append(el)
                    distance_sea_floor.append(dis)
                    
                arrival_time.extend(zero_bounces_min['time_of_arrival'].tolist())
                angle_of_departure.extend(zero_bounces_min['angle_of_departure'].tolist())
                angle_of_arrival.extend(zero_bounces_min['angle_of_arrival'].tolist())
            
            except Exception as e:
                print("wrong")

In [ ]:
import pandas as pd
print(len(distance_sea_floor))
print(len(height_DOM))
print(len(arrival_time))
print(len(angle_of_departure))
print(len(angle_of_arrival))

df = pd.DataFrame({'Distance Sea Floor [m]': distance_sea_floor, 'Height DOM [m]': height_DOM, 'Arrival Time [s]': arrival_time, 'Angle of Departure [degrees]': angle_of_departure, 'Angle of Arrival [degrees]': angle_of_arrival})

print(df)

In [ ]:
df.to_csv('TANGANYIKA_arrival_time_Bellhop_travel_distance.txt', sep='\t', index=False)

# Pylos

In [ ]:
env = pm.create_env2d()

# make the speed of sound profile for the Tanganyika site
ssp_P = ssp = [
    [   0, 1526.1641276], 
    [   25, 1523.07609121], 
    [  50, 1516.65415601], 
    [  75, 1508.87374928], 
    [ 90, 1508.18366911], 
    [ 100, 1508.12100824], 
    [ 150, 1508.79805363], 
    [ 500, 1513.91431425], 
    [1000, 1521.4932753], 
    [4000, 1565]] 

# frequency min and max: https://www.sciencedirect.com/science/article/pii/S0168900212015239
frequency_min = 25000      # 25 kHz
frequency_max = 45000      # 45 kHz

In [ ]:
# Create the 2D environment with both the source and the receiver placed 3 m above the sea floor
env = pm.create_env2d(frequency=frequency_min, min_angle=-89, max_angle=89, depth=4000, tx_depth=3997, rx_depth=3900, rx_range=10000, soundspeed=ssp_P, nbeams=10000)

# Plot the sound speed profile
pm.plot_ssp(env, width=500)
pm.plot_env(env, width=1000)

In [ ]:
# define the range of distance and elevation you want to research
distance = np.linspace(50, 10000, 200)
elevation = np.linspace(0, 1000, 11)

print(distance)
print(elevation)

In [ ]:
# initiate the lists that will be filled whith eigenray properties for each of the to be looped through situations
distance_sea_floor = []
height_DOM = []
arrival_time = []
angle_of_departure = []
angle_of_arrival = []

# loop through the distances and elevations to check for each scenario individually
for i in range(0, len(distance)):
    for j in range(0, len(elevation)):
        dis = distance[i]
        el = elevation[j]

        if el == 0:
            el = 3

        if dis == 0:
            dis = 13
        
        #print(f"dis: {dis}")
        #print(f"el: {el}")
        
        # define the depth of the ocean at the location of your transmitter
        transmitter_depth = 3997
        receiver_depth = (4000 - el)
        tx_depth = transmitter_depth - 3
        rx_depth = receiver_depth
        #print(f"receiver_depth: {receiver_depth}")
        #print(f"tx_depth: {tx_depth}")
        #print(f"rx_depth: {rx_depth}")
        
        # create two environments with different frequencies
        env_min = pm.create_env2d(frequency=frequency_min, min_angle=-89, max_angle=89, depth=4000, tx_depth=tx_depth, rx_depth=rx_depth, rx_range=dis, soundspeed=ssp_P, nbeams=100000)
        env_max = pm.create_env2d(frequency=frequency_max, min_angle=-89, max_angle=89, depth=4000, tx_depth=tx_depth, rx_depth=rx_depth, rx_range=dis, soundspeed=ssp_P, nbeams=100000)

        #pm.plot_env(env_min, surface_color='dodgerblue', bottom_color='peru', rx_color='blue', rx_plot=None, width=900)
        
        if (dis != 0) or (el != 0):
            try:
                # compute the arrivals
                arrivals_min = pm.compute_arrivals(env_min)
                arrivals_max = pm.compute_arrivals(env_max)
    
                # check which arrivals have 0 bounces against the surface or the bottom
                zero_bounces_min = arrivals_min[(arrivals_min['surface_bounces'] == 0) & (arrivals_min['bottom_bounces'] == 0)]
                zero_bounces_max = arrivals_max[(arrivals_max['surface_bounces'] == 0) & (arrivals_max['bottom_bounces'] == 0)]
    
                # count how many rays arrive with no bounces and save necessary quantities to list
                for time in zero_bounces_min['time_of_arrival']:
                    height_DOM.append(el)
                    distance_sea_floor.append(dis)
                    
                arrival_time.extend(zero_bounces_min['time_of_arrival'].tolist())
                angle_of_departure.extend(zero_bounces_min['angle_of_departure'].tolist())
                angle_of_arrival.extend(zero_bounces_min['angle_of_arrival'].tolist())
            
            except Exception as e:
                print("wrong")

In [ ]:
import pandas as pd
print(len(distance_sea_floor))
print(len(height_DOM))
print(len(arrival_time))
print(len(angle_of_departure))
print(len(angle_of_arrival))

df = pd.DataFrame({'Distance Sea Floor [m]': distance_sea_floor, 'Height DOM [m]': height_DOM, 'Arrival Time [s]': arrival_time, 'Angle of Departure [degrees]': angle_of_departure, 'Angle of Arrival [degrees]': angle_of_arrival})

print(df)

In [ ]:
df.to_csv('PYLOS_arrival_time_Bellhop_travel_distance.txt', sep='\t', index=False)

# Plotting

In [ ]:
import matplotlib.pyplot as plt

# define custom colors for each value
colors = ['darkorange', 'green', 'darkgreen']

# create a colormap from the custom colors
cmap = plt.matplotlib.colors.ListedColormap(colors)

# create a plot minimum frequency
plt.figure(figsize=(10, 8))
plt.imshow(num_zero_bounces_min.T, cmap=cmap, interpolation='nearest')

# add color bar
cbar = plt.colorbar(ticks=[0, 1, 2])
cbar.set_ticklabels(['0 (Orange)', '1 (Green)', '2 (Darkgreen)'])

# set axis labels and ticks
plt.xlabel('Distance [m]')
plt.ylabel('Elevation [m]')

plt.xticks(np.arange(0, num_zero_bounces_min.shape[0], 5), distance.astype(int)[::5])
plt.yticks(np.arange(0, num_zero_bounces_min.shape[1], 5), elevation.astype(int)[::5])

# invert the y axis
plt.gca().invert_yaxis()

# show plot
plt.title(f'Zero Bounce Arrivals for f = {frequency_min} Hz')
plt.grid(False)

# save the plot as an image file
plt.savefig('ARCA_25000Hz_extreme_elevation.png')

plt.show()

####################################################################################################################################

# create a plot maximum frequency
plt.figure(figsize=(10, 8))
plt.imshow(num_zero_bounces_max.T, cmap=cmap, interpolation='nearest')

# add color bar
cbar = plt.colorbar(ticks=[0, 1, 2])
cbar.set_ticklabels(['0 (Orange)', '1 (Green)', '2 (Darkgreen)'])

# set axis labels and ticks
plt.xlabel('Distance [m]')
plt.ylabel('Elevation [m]')

plt.xticks(np.arange(0, num_zero_bounces_min.shape[0], 5), distance.astype(int)[::5])
plt.yticks(np.arange(0, num_zero_bounces_min.shape[1], 5), elevation.astype(int)[::5])

# invert the y axis
plt.gca().invert_yaxis()

# show plot
plt.title(f'Zero Bounce Arrivals for f = {frequency_max} Hz')
plt.grid(False)

# save the plot as an image file
plt.savefig('ARCA_45000Hz_extreme_elevation.png')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# define custom colors for each value
colors = ['darkorange', 'green', 'darkgreen']

# create a colormap from the custom colors
cmap = plt.matplotlib.colors.ListedColormap(colors)

# create a plot minimum frequency
plt.figure(figsize=(10, 8))
plt.imshow(num_zero_bounces_min.T, cmap=cmap, interpolation='nearest')

# add color bar
cbar = plt.colorbar(ticks=[0, 1, 2])
cbar.set_ticklabels(['0 (Orange)', '1 (Green)', '2 (Darkgreen)'])

# set axis labels and ticks
plt.xlabel('Hill Height [m]')
plt.ylabel('Elevation [m]')

plt.xticks(np.arange(0, num_zero_bounces_min.shape[0], 8), hill_height.astype(int)[::5])
plt.yticks(np.arange(0, num_zero_bounces_min.shape[1], 5), elevation.astype(int)[::5])

# invert the y axis
plt.gca().invert_yaxis()

# show plot
plt.title(f'Zero Bounce Arrivals for f = {frequency_min} Hz')
plt.grid(False)

# save the plot as an image file
plt.savefig('ARCA_25000Hz_Hill_Height.png')

plt.show()

####################################################################################################################################

# create a plot maximum frequency
plt.figure(figsize=(10, 8))
plt.imshow(num_zero_bounces_max.T, cmap=cmap, interpolation='nearest')

# add color bar
cbar = plt.colorbar(ticks=[0, 1, 2])
cbar.set_ticklabels(['0 (Orange)', '1 (Green)', '2 (Darkgreen)'])

# set axis labels and ticks
plt.xlabel('Hill Height [m]')
plt.ylabel('Elevation [m]')

plt.xticks(np.arange(0, num_zero_bounces_max.shape[0], 8), hill_height.astype(int)[::5])
plt.yticks(np.arange(0, num_zero_bounces_max.shape[1], 5), elevation.astype(int)[::5])

# invert the y axis
plt.gca().invert_yaxis()

# show plot
plt.title(f'Zero Bounce Arrivals for f = {frequency_max} Hz')
plt.grid(False)

# save the plot as an image file
plt.savefig('ARCA_45000Hz_Hill_Height.png')

plt.show()